In [127]:
# !pip install river

In [128]:
# !pip install -U numpy

In [129]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

In [130]:
df = pd.read_csv("train_all.csv")

In [131]:
df.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10
0,365.684615,380.192308,22,40,44,4,5,69,61,64,47
1,396.432836,386.828358,25,32,33,1,5,77,66,57,50
2,335.674157,374.764045,26,46,51,5,6,108,88,76,56
3,346.833333,385.871795,27,40,47,7,5,103,71,65,52
4,371.468750,395.756250,26,44,50,6,6,94,63,74,56


A0) Position in X axis => position X [1, 2, 3, 2, 1] / 5  
A1) Position in Y axis => position Y [200, 150, 130, 170] / 4  
A2) Number of coins collected => Total  
A3) Number of destroyed enemies => Total  
A4) Number of shots => Total  
A5) Number of shots without enemies => Total (A4 - A3)  
A6) Level reach  
A7) key X pressed count  
A8) key Y pressed count  
A9) Number of enemy created  
A10) Number of coin created  

In [132]:
# select columns
df = df[['A2','A3','A4','A6','A7','A8','A9','A10']]

In [133]:
df.head()

,A2,A3,A4,A6,A7,A8,A9,A10
0,22,40,44,5,69,61,64,47
1,25,32,33,5,77,66,57,50
2,26,46,51,6,108,88,76,56
3,27,40,47,5,103,71,65,52
4,26,44,50,6,94,63,74,56


In [134]:
from river import cluster
from river import stream
from river import tree
from river import evaluate
from river import metrics

streamkmeans = cluster.STREAMKMeans(n_clusters=4, halflife=0.5, sigma=1.5, seed=0)

In [135]:
dt_scaler = StandardScaler();
scaled_features = dt_scaler.fit_transform(df.values)
scaled_features_df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

In [136]:
for x in scaled_features_df.iterrows():
  data = {}
  for y in range(x[1].size):
    data[y] = x[1][y]
  streamkmeans = streamkmeans.learn_one(data)

In [137]:
streamkmeans.centers

{0: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x0000021A7FDC26F0>>, 0, 1.5),
             {0: 0.7322735843427552,
              1: 1.3372265031263773,
              2: 1.5548500935186236,
              3: 0.6397382437992175,
              4: 0.6398957941588337,
              5: 1.7243626400475878,
              6: 1.1941929575033665,
              7: 0.798335314517215}),
 1: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x0000021A7FDC26F0>>, 0, 1.5),
             {0: -1.1536842569002363,
              1: -1.1847968339714865,
              2: -1.051713317928328,
              3: -1.5068533097238896,
              4: -1.3290396725203224,
              5: -0.7831962909716854,
              6: -1.2804612696439,
              7: -1.4716234767172405}),
 2: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x0000021A7FDC26F0>>, 0, 1.5),
             {0: 1.775977661209389,
     

In [138]:
classes = []
for x in scaled_features_df.iterrows():
  data = {}
  for y in range(x[1].size):
    data[y] = x[1][y]

  classes.append(streamkmeans.predict_one(data))
df['class'] = classes

In [139]:
model = tree.HoeffdingAdaptiveTreeClassifier(
  grace_period=100,
  split_confidence=1e-5,
  leaf_prediction='nb',
  nb_threshold=10,
  seed=0
)

In [140]:
train_datas = []
for x in list(df.iterrows()):
  data = {}
  for y in range(x[1].size-1):
    data[y] = x[1][y]
  model.learn_one(data, x[1][-1])

In [149]:
joblib.dump(model, 'model2.h5')

['model2.h5']

In [142]:
df['A2/A10'] = (df['A2']/df['A10'])*100
df['A3/A9'] = (df['A3']/df['A9'])*100

In [143]:
collect_coin_ratio = list(df.groupby(['class']).mean()['A2/A10'])
enemy_killed_retio = list(df.groupby(['class']).mean()['A3/A9'])
print(collect_coin_ratio)
print(enemy_killed_retio)


[40.704984954921095, 34.15399733253759, 52.80479447413516, 30.49181347833899]
[71.67469884222716, 45.19751302001099, 62.738253087889824, 62.91270029124603]


In [144]:
sum_df = pd.DataFrame()
# sum_df['A0'] = list(df.groupby(['class']).mean()['A0'])
# sum_df['A1'] = list(df.groupby(['class']).mean()['A1'])
sum_df['A2'] = list(df.groupby(['class']).mean()['A2'])
sum_df['A3'] = list(df.groupby(['class']).mean()['A3'])
sum_df['A4'] = list(df.groupby(['class']).mean()['A4'])
# sum_df['A5'] = list(df.groupby(['class']).mean()['A5'])
sum_df['A6'] = list(df.groupby(['class']).mean()['A6'])
sum_df['A7'] = list(df.groupby(['class']).mean()['A7'])
sum_df['A8'] = list(df.groupby(['class']).mean()['A8'])
sum_df['A9'] = list(df.groupby(['class']).mean()['A9'])
sum_df['A10'] = list(df.groupby(['class']).mean()['A10'])


In [145]:
sum_df

,A2,A3,A4,A6,A7,A8,A9,A10
0,19.896907,72.814433,89.783505,5.494845,105.989691,56.319588,100.082474,47.371134
1,5.471910,11.797753,19.269663,1.853933,22.494382,11.426966,21.000000,14.730337
2,20.285714,31.535714,44.678571,3.571429,53.928571,43.250000,49.178571,38.142857
3,9.152174,38.173913,49.717391,4.043478,85.456522,17.478261,58.282609,29.347826


In [148]:
labels = []
num_labels = len(collect_coin_ratio)
for i in range(num_labels):
    if i % 2 == 0:
        sorted_index = np.argsort(collect_coin_ratio)
    else:
        sorted_index = np.argsort(enemy_killed_retio)

    sorted_index = list(sorted_index)
    sorted_index.reverse()
    for max_index in sorted_index:
        if max_index not in labels:
            labels.append(max_index)
            break

print(labels)

LABELS = {
    labels[0]: 'Hardcore Achiever',
    labels[1]: 'Hardcore Killer',
    labels[2]: 'Casual Achiever',
    labels[3]: 'Casual Killer',
}
print(LABELS)

[2, 0, 1, 3]
{2: 'Hardcore Achiever', 0: 'Hardcore Killer', 1: 'Casual Achiever', 3: 'Casual Killer'}
